# Folium의 Heatmap  
---  

### 개요  
- folium 모듈을 사용하요 만든 HeatMap

### 목차  
1. 모듈 로드
2. HeatMap


---
## 1. 모듈 로드

In [64]:
import pandas as pd
import numpy as np
import json

from pprint import pprint as pp

import folium 
from folium import plugins
from folium.plugins import HeatMap

---
## 5. HeatMap

In [72]:
df_PatientInfo = pd.read_csv('/home/seong/workspace/folium_study/data/PatientInfo.csv')
df_region = pd.read_csv('/home/seong/workspace/folium_study/data/Region.csv')

In [73]:
index_null= df_PatientInfo.contact_number[df_PatientInfo.contact_number == "-"].index
df_patient_nn= df_PatientInfo.drop(index_null)
# df_patient_nn

df_patient_nn["contact_number"]= pd.to_numeric(df_patient_nn.contact_number)
del df_patient_nn["contact_number"]
# df_patient_nn.info()

df_marge= pd.merge(df_patient_nn, df_region, on= "city")
# pp(type(df_marge))
# df_marge

df_group= df_marge.groupby("city").count()
# pp(type(df_group))
df_group= {"patient_id": df_group.patient_id}
df_group= pd.DataFrame(df_group)

# df_group

df_city= df_marge
df_city= {"city": df_city.city, "latitude": df_city.latitude, "longitude": df_city.longitude}
df_city= pd.DataFrame(df_city)
df_city_nn= df_city.drop_duplicates("city")

# df_city_nn

df_city_nn= pd.merge(df_city_nn, df_group, on= "city")
# df_city_nn

df_city_nn= df_city_nn.reset_index().sort_values("city")
# df_city_nn

df_city_xy= {"latitude": list(df_city_nn.latitude), "longitude": list(df_city_nn.longitude), "patient_id": list(df_city_nn.patient_id)}
df_city_xy= pd.DataFrame(df_city_xy)
df_city_xy

df_patient = pd.merge(df_PatientInfo[['patient_id','province','city']],
                     df_region[['province','city','latitude','longitude']], 
                     how = 'left', 
                     on = ['province','city'])
regional_count = df_patient[['latitude','longitude']].dropna()

In [74]:
#코로나 데이터 사용

m = folium.Map(location= [36, 128], zoom_start= 7, tiles='Openstreetmap')

folium.plugins.HeatMap(df_city_xy, radius= 30).add_to(m)

m.save('heatmap.html')

m